# Applying EMD to de-trend historical consumption
EMD (empirical mode decomposition) is an algorithm to decompose a time serie into intrisic mode functions (IMFs). The EMD algorithm is also called Hilbert-Huang transform, and is robust to non stationary and non linear time series. Original paper : https://royalsocietypublishing.org/doi/10.1098/rspa.1998.0193

### Set up the folder where the data lives

In [ ]:
data_dir = 'C:\\Users\\chris\\Documents\\GitHub\\data\\f28e28af-5e8b-44e4-bf01-e85eb1c221fb\\f28e28af-5e8b-44e4-bf01-e85eb1c221fb\\load'

In [ ]:
#get all the files living in the data folder
from os import listdir
from os.path import isfile, join
onlyfiles = [f for f in listdir(data_dir) if isfile(join(data_dir, f))]
print(onlyfiles[0])

In [ ]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime,timezone

### Read two random files from the dataset
Read the dates using the data parser, and transforming to UTC (needed as +01 and +02 co-exist in the same file)

In [ ]:
rnd = np.random.randint(0,len(onlyfiles),2)
pd1 = pd.read_csv(join(data_dir,onlyfiles[rnd[0]]),sep=';',parse_dates=[0],date_parser=lambda col: pd.to_datetime(col, utc=True))
pd2 = pd.read_csv(join(data_dir,onlyfiles[rnd[1]]),sep=';',parse_dates=[0],date_parser=lambda col: pd.to_datetime(col, utc=True))

In [ ]:
print(f' -- Analyzing {onlyfiles[rnd[0]]} as first example')
print(f' -- Analyzing {onlyfiles[rnd[1]]} as second example')

In [ ]:
# check types
pd1.dtypes

In [ ]:
# look at the first four rows
pd1.head(4)

### Set the dates as index and fix the summer winter stuff

In [ ]:
pd1=pd1.set_index('date_tz')
pd2=pd2.set_index('date_tz')

In [ ]:
#fixing the double hour (from summer to winter)
pd1[pd1.index==datetime(2017,10,29,2,0,0,tzinfo=timezone.utc)]=pd1[pd1.index==datetime(2017,10,29,2,0,0,tzinfo=timezone.utc)]/2
pd1[pd1.index==datetime(2018,10,28,2,0,0,tzinfo=timezone.utc)]=pd1[pd1.index==datetime(2018,10,28,2,0,0,tzinfo=timezone.utc)]/2

In [ ]:
# remove duplicated hours (from winter to summer)
pd1 = pd1[~pd1.index.duplicated()]
pd2 = pd2[~pd2.index.duplicated()]

### First de-trending : remove the average over the whole period
Not sure this step is necessary

In [ ]:
m1 = pd1['kWh/h'].mean()
m2 = pd2['kWh/h'].mean()
pd1['kWh/h'] = pd1['kWh/h']-m1
pd2['kWh/h'] = pd2['kWh/h']-m2
print(f' -- Removed a constant of {m1:0.2f} kWh/h from the first example')
print(f' -- Removed a constant of {m2:0.2f} kWh/h from the first example')

In [ ]:
pd1.plot(figsize=[8,8])
pd2.plot(figsize=[8,8])

###  Resample the hourly curves into weekly curves

In [ ]:
pd1_rs = pd1.resample(rule='W',axis=0).mean()
pd2_rs = pd2.resample(rule='W',axis=0).mean()

In [ ]:
pd1_rs.plot(figsize=[8,8])
pd2_rs.plot(figsize=[8,8])

## Get the seasonal trend using EMD on the sub sampled weekly timeseries
Should test EEMD as well, but more computationally expensive

In [ ]:
from PyEMD import EMD,Visualisation
emd1 = EMD()
emd2 = EMD()

## Get the IMFs with, 10 iterations per IMFs

In [ ]:
emd1.FIXE = 10
imfs1 = emd1(pd1_rs['kWh/h'])
emd2.FIXE = 10
imfs2 = emd2(pd2_rs['kWh/h'])

In [ ]:
v1=Visualisation()
v1.plot_imfs(imfs=imfs1,include_residue=False)
v2=Visualisation()
v2.plot_imfs(imfs=imfs2,include_residue=False)

We can see that the seasonal trends are captured. The difficulty with EMD is to decide (programatically) which are the IMFs which embed the seasonal info (four seasons), and which are the ones with higher frequence.

### Add the IMFs in a Dataframe and merge with the sub sampled load

In [ ]:
imfs1_pd=pd.DataFrame.from_records(np.transpose(imfs1),index=pd1_rs.index,columns=[f'imf{n}' for n in range(0,imfs1.shape[0])])
imfs2_pd=pd.DataFrame.from_records(np.transpose(imfs2),index=pd2_rs.index,columns=[f'imf{n}' for n in range(0,imfs2.shape[0])])

In [ ]:
pd1_new = pd1_rs.merge(right=imfs1_pd,left_index=True,right_index=True)
pd2_new = pd2_rs.merge(right=imfs2_pd,left_index=True,right_index=True)

In [ ]:
pd1_new.plot(figsize=[8,8])
pd2_new.plot(figsize=[8,8])

## Upsample the IMFs to hourly frequence
Linear interpolation is more than OK here, no need to complicate things.

In [ ]:
imfs1_pd_hr = imfs1_pd.resample(rule='H',axis=0).interpolate(method='linear')
imfs2_pd_hr = imfs2_pd.resample(rule='H',axis=0).interpolate(method='linear')

Merge with the original timeserie

In [ ]:
pd1=pd1.merge(right=imfs1_pd_hr,left_index=True,right_index=True)
pd2=pd2.merge(right=imfs2_pd_hr,left_index=True,right_index=True)

In [ ]:
pd1.plot(figsize=[8,8])
pd2.plot(figsize=[8,8])

## Do the de-trending by removing the 3 first IMFs from the timeserie
The number three is completly experimental, and might not fit certain timeseries that have few IMFs. A bit smarter approach (could be as simple as if len(IMF)==3, use one, if len(IMF)==5, use two, etc...) should be used here.

In [ ]:
pd1['detrend']=pd1['kWh/h']-(pd1[f'imf{imfs1.shape[0]-1}']+pd1[f'imf{imfs1.shape[0]-2}']+pd1[f'imf{imfs1.shape[0]-3}'])
pd2['detrend']=pd2['kWh/h']-(pd2[f'imf{imfs2.shape[0]-1}']+pd2[f'imf{imfs2.shape[0]-2}']+pd2[f'imf{imfs2.shape[0]-3}'])

In [ ]:
pd1.plot(figsize=[12,8])
pd2.plot(figsize=[12,8])

### Final standardisation : put all the values between zero and one

In [ ]:
pd1['detrend_norm']=(pd1['detrend']-pd1['detrend'].min())/(pd1['detrend'].max()-pd1['detrend'].min())
pd2['detrend_norm']=(pd2['detrend']-pd2['detrend'].min())/(pd2['detrend'].max()-pd2['detrend'].min())

In [ ]:
pd1['detrend_norm'].plot(figsize=[12,8])
pd2['detrend_norm'].plot(figsize=[12,8])

## Use Plotly for visualisation

In [ ]:
import plotly.graph_objects as go

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=pd1.index, y=pd1['detrend_norm'], name="Detrend Norm 1",
                         line_color='deepskyblue'))

fig.add_trace(go.Scatter(x=pd2.index, y=pd2['detrend_norm'], name="Detrend Norm 2",
                         line_color='dimgray'))

fig.update_layout(title_text='Detrend with Rangeslider',
                  xaxis_rangeslider_visible=True)
fig.show()

## Starting looking at some trends
Here I look at the average for each day

In [ ]:
fig = plt.figure(figsize=[16,8])
for d in range(0,7):
    test_pd = pd1.loc[pd1.index.dayofweek==d,'detrend_norm']
    mean_cons=[]
    for f in range(0,24):
        mean_cons.append(test_pd.loc[(test_pd.index.hour==f)].mean())
    plt.plot(range(0,24),mean_cons)

fig2 = plt.figure(figsize=[16,8])
for d in range(0,7):
    test_pd = pd2.loc[pd2.index.dayofweek==d,'detrend_norm']
    mean_cons=[]
    for f in range(0,24):
        mean_cons.append(test_pd.loc[(test_pd.index.hour==f)].mean())
    plt.plot(range(0,24),mean_cons)